Simple notebook de prediction du score DPE a partir d'un dataset de l'ADEME
disponible sur https://data.ademe.fr/datasets/dpe-v2-tertiaire-2

In [1]:
import pandas as pd
pd.options.display.max_columns = 100
pd.options.display.max_rows = 60
pd.options.display.max_colwidth = 100
pd.options.display.precision = 10
pd.options.display.width = 160
pd.set_option("display.float_format", "{:.4f}".format)
import numpy as np
import re
import typing as t
from collections import Counter
from sklearn.preprocessing import OrdinalEncoder
import json
import datetime


In [2]:
url = "https://data.ademe.fr/data-fair/api/v1/datasets/dpe-v2-tertiaire-2/lines?size=10000&format=csv&after=10000%2C965634&header=true"

data = pd.read_csv(url)
print(data.shape)

(10000, 63)


In [3]:
print(data.head())

           N°DPE Date_réception_DPE Date_établissement_DPE Date_visite_diagnostiqueur                 Modèle_DPE N°_DPE_remplacé Date_fin_validité_DPE  \
0  2369T3957539J         2023-11-21             2023-11-20                 2023-11-05  DPE 2006 tertiaire et ERP   2369T3818150G            2033-11-19   
1  2302T0483966V         2023-02-14             2023-02-09                 2023-02-09  DPE 2006 tertiaire et ERP             NaN            2033-02-08   
2  2111T0471835N         2021-10-10             2021-10-09                 2021-10-07  DPE 2006 tertiaire et ERP             NaN            2031-10-08   
3  2130T0774038L         2021-11-25             2021-11-24                 2021-11-23  DPE 2006 tertiaire et ERP             NaN            2031-11-23   
4  2245T0183882T         2022-01-31             2022-01-30                 2022-01-19  DPE 2006 tertiaire et ERP             NaN            2032-01-29   

   Version_DPE N°_DPE_immeuble_associé         Méthode_du_DPE N°_immatricul

In [4]:
def rename_columns(columns: t.List[str]) -> t.List[str]:
    columns = [col.lower() for col in columns]

    rgxs = [
        (r"[°|/|']", "_"),
        (r"²", "2"),
        (r"[(|)]", ""),
        (r"é|è", "e"),
        (r"_+", "_"),
    ]
    for rgx in rgxs:
        columns = [re.sub(rgx[0], rgx[1], col) for col in columns]

    return columns


In [5]:
    """
    Simplifier le nom des colonnes
    """

    data.columns = rename_columns(data.columns)



In [6]:
    """
    rm missing target
    """
    target = "etiquette_dpe"
    data.dropna(subset=target, inplace=True)

    """
    categorical columns
    """


'\ncategorical columns\n'

In [7]:
    columns_categorical = [
        "version_dpe",
        "methode_du_dpe",
        "categorie_erp",
        "secteur_activite",
        "type_energie_principale_chauffage",
        "type_energie_n_1",
        "type_energie_n_2",
        "type_energie_n_3",
        "type_usage_energie_n_1",
        "type_usage_energie_n_2",
        "type_usage_energie_n_3",
    ]
    for col in columns_categorical:
        data[col].fillna("non renseigné", inplace=True)


In [8]:
    """
    regroup type energies
    """
    type_energie = []
    for col in [
        "type_energie_principale_chauffage",
        "type_energie_n_1",
        "type_energie_n_2",
        "type_energie_n_3",
    ]:
        type_energie += list(data[col])
    type_energie_count = Counter(type_energie)
    print(type_energie_count.most_common(20))

    type_energie_map = {
        "non renseigné": "non renseigné",
        "Électricité": "Électricité",
        "Électricité d'origine renouvelable utilisée dans le bâtiment": "Électricité",
        "Gaz naturel": "Gaz naturel",
        "Butane": "GPL",
        "Propane": "GPL",
        "GPL": "GPL",
        "Fioul domestique": "Fioul domestique",
        "Réseau de Chauffage urbain": "Réseau de Chauffage urbain",
        "Charbon": "Combustible fossile",
        "autre combustible fossile": "Combustible fossile",
        "Bois – Bûches": "Bois",
        "Bois – Plaquettes forestières": "Bois",
        "Bois – Granulés (pellets) ou briquettes": "Bois",
        "Bois – Plaquettes d’industrie": "Bois",
    }

    for col in [
        "type_energie_principale_chauffage",
        "type_energie_n_1",
        "type_energie_n_2",
        "type_energie_n_3",
    ]:
        data[col] = data[col].apply(lambda d: type_energie_map[d])

    type_energie = []
    for col in [
        "type_energie_principale_chauffage",
        "type_energie_n_1",
        "type_energie_n_2",
        "type_energie_n_3",
    ]:
        type_energie += list(data[col])

    type_energie_count = Counter(type_energie)
    print(type_energie_count.most_common(20))



[('non renseigné', 12781), ('Électricité', 7351), ('Gaz naturel', 1691), ('Fioul domestique', 211), ('Réseau de Chauffage urbain', 165), ('Propane', 62), ('autre combustible fossile', 52), ('Bois – Granulés (pellets) ou briquettes', 34), ('Bois – Bûches', 28), ('GPL', 24), ("Électricité d'origine renouvelable utilisée dans le bâtiment", 10), ('Bois – Plaquettes forestières', 7), ('Bois – Plaquettes d’industrie', 4), ('Butane', 3), ('Charbon', 1)]
[('non renseigné', 12781), ('Électricité', 7361), ('Gaz naturel', 1691), ('Fioul domestique', 211), ('Réseau de Chauffage urbain', 165), ('GPL', 89), ('Bois', 73), ('Combustible fossile', 53)]


In [9]:
    """
    regroup type usage
    """
    type_usage = []
    for col in [
        "type_usage_energie_n_1",
        "type_usage_energie_n_2",
        "type_usage_energie_n_3",
    ]:
        type_usage += list(data[col])

    type_usage_count = Counter(type_usage)
    print(type_usage_count.most_common(20))

    type_usage_map = {
        "non renseigné": "non renseigné",
        "périmètre de l'usage inconnu": "non renseigné",
        "Chauffage": "Chauffage",
        "Eau Chaude sanitaire": "Eau Chaude sanitaire",
        "Eclairage": "Eclairage",
        "Refroidissement": "Refroidissement",
        "Ascenseur(s)": "Ascenseur(s)",
        "auxiliaires et ventilation": "Refroidissement",
        "Autres usages": "Autres usages",
        "Bureautique": "Autres usages",
        "Abonnements": "Autres usages",
        "Production d'électricité à demeure": "Autres usages",
    }
    for col in [
        "type_usage_energie_n_1",
        "type_usage_energie_n_2",
        "type_usage_energie_n_3",
    ]:
        data[col] = data[col].apply(lambda d: type_usage_map[d])

    type_usage = []
    for col in [
        "type_usage_energie_n_1",
        "type_usage_energie_n_2",
        "type_usage_energie_n_3",
    ]:
        type_usage += list(data[col])

    type_usage_count = Counter(type_usage)
    print(type_usage_count.most_common(20))



[('non renseigné', 8177), ("périmètre de l'usage inconnu", 6032), ('Chauffage', 1391), ('Eau Chaude sanitaire', 656), ('Eclairage', 321), ('Autres usages', 77), ('Refroidissement', 68), ('Ascenseur(s)', 63), ('Bureautique', 12), ('auxiliaires et ventilation', 11), ("Production d'électricité à demeure", 10)]
[('non renseigné', 14209), ('Chauffage', 1391), ('Eau Chaude sanitaire', 656), ('Eclairage', 321), ('Autres usages', 99), ('Refroidissement', 79), ('Ascenseur(s)', 63)]


In [10]:
    """
    Encode categorical columns
    """
    encoder = OrdinalEncoder()

    data[columns_categorical] = encoder.fit_transform(data[columns_categorical])
    for col in columns_categorical:
        data[col] = data[col].astype(int)

    # save categorical mappings
    mappings = {}
    for i, col in enumerate(encoder.feature_names_in_):
        mappings[col] = {
            int(value): category
            for value, category in enumerate(encoder.categories_[i])
        }

    # Save the mappings to a JSON file
    with open("./categorical_mappings.json", "w", encoding="utf-8") as f:
        json.dump(mappings, f, ensure_ascii=False, indent=4)


In [11]:
    """
    date de visite
    """
    columns_dates = ["date_visite_diagnostiqueur"]

    # supprimer le jour, garder annee et mois comme float
    col = "date_visite_diagnostiqueur"
    data[col] = data[col].apply(lambda d: float(".".join(d.split("-")[:2])))



In [12]:
    """
    Floats
    """
    columns_float = [
        "conso_kwhep_m2_an",
        "emission_ges_kgco2_m2_an",
        "surface_utile",
        "conso_e_finale_energie_n_1",
        "conso_e_primaire_energie_n_1",
        "frais_annuel_energie_n_1",
        "conso_e_finale_energie_n_2",
        "conso_e_primaire_energie_n_2",
        "frais_annuel_energie_n_2",
        "conso_e_finale_energie_n_3",
        "conso_e_primaire_energie_n_3",
        "frais_annuel_energie_n_3",
    ]

    for col in columns_float:
        data[col].fillna(0.0, inplace=True)

    # add
    data["conso_finale_energie"] = 0.0
    data["conso_primaire_energie"] = 0.0
    data["frais_annuel_energie"] = 0.0
    for n in range(1, 4):
        col = f"conso_e_finale_energie_n_{n}"
        data["conso_finale_energie"] += data[col]
        col = f"conso_e_primaire_energie_n_{n}"
        data["conso_primaire_energie"] += data[col]
        col = f"frais_annuel_energie_n_{n}"
        data["frais_annuel_energie"] += data[col]

    """
    columns int
    """
    columns_int = [
        "annee_construction",
        "nombre_occupant",
        "n_etage_appartement",
    ]

    for col in columns_int:
        data[col].fillna(-1, inplace=True)
        data[col] = data[col].astype(int)


In [13]:
    """
    target : etiquette SPE
    """
    target_encoding = {"A": 1, "B": 2, "C": 3, "D": 4, "E": 5, "F": 6, "G": 7}
    data[target] = data[target].apply(lambda d: target_encoding[d])


In [14]:
    # final feature set
    features = [
        "n_dpe",
        "version_dpe",
        "methode_du_dpe",
        "categorie_erp",
        "secteur_activite",
        "type_energie_principale_chauffage",
        "type_energie_n_1",
        "type_usage_energie_n_1",
        "conso_kwhep_m2_an",
        "emission_ges_kgco2_m2_an",
        "surface_utile",
        "conso_finale_energie",
        "conso_primaire_energie",
        "frais_annuel_energie",
        "annee_construction",
        target,
    ]

    data = data[features].copy()
    data.reset_index(inplace=True, drop=True)

    # save to file
    output_file = (
        f"./dpe_tertiaire_{datetime.datetime.now().strftime('%Y%m%d')}.csv"
    )


    data.to_csv(output_file, index=False)
    print(f"data saved to {output_file}")


data saved to ./dpe_tertiaire_20240314.csv


# Training



In [15]:
from sklearn.model_selection import train_test_split, GridSearchCV, KFold
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report, accuracy_score, roc_auc_score


In [16]:
# load data
input_file = output_file
data = pd.read_csv(input_file)
# shuffle
data = data.sample(frac=1, random_state=808).reset_index(drop=True)


In [17]:
    # split
    # Assuming the last column is the target variable
    X = data.iloc[:, :-1]  # Features
    y = data.iloc[:, -1]  # Target variable
    assert y.name == "etiquette_dpe"
    # id = list(X.n_dpe)
    # Split the data into training and testing sets
    X_train, X_test, y_train, y_test = train_test_split(
        X, y, test_size=0.2, random_state=808
    )

    X_train.drop(columns=["n_dpe"], inplace=True)
    id_test = list(X_test.n_dpe)
    X_test.drop(columns=["n_dpe"], inplace=True)


In [18]:
    # Initialize the model
    rf = RandomForestClassifier()


In [19]:
    # Define the parameter grid
    param_grid = {
        "n_estimators": [200, 300],  # Number of trees
        "max_depth": [10],  # Maximum depth of the trees
        "min_samples_leaf": [1, 5],  # Maximum depth of the trees
    }

    # Setup GridSearchCV with k-fold cross-validation
    cv = KFold(n_splits=3, random_state=84, shuffle=True)

    grid_search = GridSearchCV(
        estimator=rf, param_grid=param_grid, cv=cv, scoring="accuracy", verbose=1
    )


In [20]:
    # Fit the model
    grid_search.fit(X_train, y_train)


Fitting 3 folds for each of 4 candidates, totalling 12 fits


GridSearchCV(cv=KFold(n_splits=3, random_state=84, shuffle=True),
             estimator=RandomForestClassifier(),
             param_grid={'max_depth': [10], 'min_samples_leaf': [1, 5],
                         'n_estimators': [200, 300]},
             scoring='accuracy', verbose=1)

In [21]:
    # Best parameters and best score
    print(f"Best parameters: {grid_search.best_params_}")
    print(f"Best cross-validation score: {grid_search.best_score_}")
    print(f"Best model: {grid_search.best_estimator_}")

    # Evaluate on the test set
    yhat = grid_search.predict(X_test)
    print(classification_report(y_test, yhat))



Best parameters: {'max_depth': 10, 'min_samples_leaf': 1, 'n_estimators': 300}
Best cross-validation score: 0.8229251453976442
Best model: RandomForestClassifier(max_depth=10, n_estimators=300)
              precision    recall  f1-score   support

           1       0.99      0.89      0.93       105
           2       0.84      0.85      0.84       121
           3       0.92      0.82      0.87       291
           4       0.81      0.83      0.82       270
           5       0.68      0.86      0.76       147
           6       0.87      0.75      0.81        64
           7       0.89      0.91      0.90       124

    accuracy                           0.84      1122
   macro avg       0.86      0.84      0.85      1122
weighted avg       0.85      0.84      0.85      1122



In [22]:
# regroup into predictions dataframe
probabilities = grid_search.predict_proba(X_test)

predictions = pd.DataFrame()
predictions["id"] = id_test
predictions["prob"] = np.max(probabilities, axis=1)
predictions["yhat"] = yhat
predictions["y"] = y_test.values
print(predictions.head())


              id   prob  yhat  y
0  2113T0707016A 1.0000     1  1
1  2373T4274354S 0.7377     4  4
2  2142T1010760P 0.5169     3  3
3  2244T0873273T 0.4379     5  4
4  2294T1585219F 0.9309     3  3


In [23]:
    # feature importance
    feature_importances = grid_search.best_estimator_.feature_importances_
    feature_names = X_train.columns

    # Create a dictionary mapping feature names to their importance
    importance_dict = dict(zip(feature_names, feature_importances))
    importance_dict = dict(
        sorted(importance_dict.items(), key=lambda item: item[1], reverse=True)
    )

    print(importance_dict)



{'conso_kwhep_m2_an': 0.4560004648582145, 'emission_ges_kgco2_m2_an': 0.2356980533905615, 'conso_finale_energie': 0.06646916976942052, 'conso_primaire_energie': 0.06574298220460331, 'frais_annuel_energie': 0.046111311278519064, 'secteur_activite': 0.03012996189963547, 'methode_du_dpe': 0.01969181062145716, 'surface_utile': 0.01968677860206445, 'annee_construction': 0.01791968885315905, 'type_energie_n_1': 0.01297045415114978, 'version_dpe': 0.011185842677147782, 'categorie_erp': 0.007694512776899341, 'type_usage_energie_n_1': 0.005752469930623638, 'type_energie_principale_chauffage': 0.00494649898654448}
